# Where do you want to eat? - By Micaela Montes
---



## Project overview and setup

Hi there! My friends and I are travelling to Peru for spring break, and want to make each meal count by going to the best restaurants.





In order to make a list of restaurants to choose from, I thought it was a good idea to make a sentiment analysis model to process and analyze customer reviews of restaurants in Peru. By doing this, I can find the most valued places to eat, and the restaurant search will be easier.


The dataset used for this project comes from Kaggle and can be found using this [link](https://www.kaggle.com/datasets/lazaro97/peruvian-food-reviews).

In [66]:
!pip install pyspark

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
!ls "/content/drive/My Drive/projects/peruvian_food/peruvian_food_dataset/"

data


In [69]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("PeruvianFoodAnalysis")\
        .getOrCreate()

## Mounting the relevant csv files:



In [70]:
# Table 1: Restaurants
restaurants_df = spark.read.csv(
    '/content/drive/My Drive/projects/peruvian_food/peruvian_food_dataset/data/restaurants.csv/',
    header=True,  # Assumes the first row is the header
    inferSchema=True  # Infers the schema of the columns based on the data
)

# Table 2: Reviews
reviews_df = spark.read.csv(
    '/content/drive/My Drive/projects/peruvian_food/peruvian_food_dataset/data/reviews.csv/',
    header=True,
    inferSchema=True
)

# Lexicons
afinn_df = spark.read.csv("/content/drive/My Drive/projects/peruvian_food/peruvian_food_dataset/data/spanish-nrc/lexico_afinn.csv", header=True, inferSchema=True)
nrc_df = spark.read.csv("/content/drive/My Drive/projects/peruvian_food/peruvian_food_dataset/data/spanish-nrc/lexico_nrc.csv", header=True, inferSchema=True)

# Show the first few rows of reviews and restaurants dataframes to verify everything looks good
restaurants_df.show(5)
reviews_df.show(5)

+------+--------------------+--------------------+----------+----------+----------+------+--------------------+-----+---------+---------+---------+-----------+
|    id|                name|                 tag|         x|         y|  district|IDDIST|           direction|stars|n_reviews|min_price|max_price|   platform|
+------+--------------------+--------------------+----------+----------+----------+------+--------------------+-----+---------+---------+---------+-----------+
| 71036| Restaurante Statera|$$$$||Peruana||co...|-12.115658| -77.04344|MIRAFLORES|150122|Avenida Mariscal ...|   50|      130|     80.0|    280.0|tripadvisor|
| 72471|   Maras Restaurante|$$$$||Peruana||La...|-12.092106| -77.02466|SAN ISIDRO|150131|Amador Merino Rey...|   45|      675|    158.0|    237.0|tripadvisor|
| 29667|Ventarrón Restaur...|$$ - $$$||Peruana...|-12.149373|-77.020706|  BARRANCO|150104|Avenida Grau 276 ...|   45|      304|     12.0|     72.0|tripadvisor|
|144186|Aji 555 Real Thai...|$$ - $$$||A

In [71]:
restaurants_df.printSchema()
reviews_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: double (nullable = true)
 |-- district: string (nullable = true)
 |-- IDDIST: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- n_reviews: string (nullable = true)
 |-- min_price: string (nullable = true)
 |-- max_price: string (nullable = true)
 |-- platform: string (nullable = true)

root
 |-- id_review: string (nullable = true)
 |-- review: string (nullable = true)
 |-- title: string (nullable = true)
 |-- score: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- id_nick: string (nullable = true)
 |-- service: string (nullable = true)
 |-- date: string (nullable = true)
 |-- platform: string (nullable = true)



Selecting Relevant Columns:

In [72]:
restaurants = restaurants_df['id', 'name', 'district']
reviews = reviews_df['id_review', 'review', 'service', 'score']

In [73]:
reviews.select("id_review").show()

+--------------------+
|           id_review|
+--------------------+
|               R1245|
|               R1246|
|               R1247|
|               R1248|
|               R1249|
|Statera no tiene ...|
|               R1250|
|Algo que me gustó...|
|               R1251|
|               R1252|
|               R1253|
|               R1254|
|               R1255|
|               R1256|
|               R1257|
|Mejorar urgente e...|
|               R1258|
|               R1259|
|               R1260|
|               R1261|
+--------------------+
only showing top 20 rows



How much information do we have in terms of reviews and number of restaurants?

In [74]:
reviews.count()

1345723

In [75]:
restaurants.count()

9871

## Cleaning the Review df

Since there are over 130,000 reviews, I can clean the dataset without worrying too much about losing important information.

In [76]:
from pyspark.sql.functions import col

# Filter to keep only rows where 'id_review' starts with 'R'
filtered_reviews = reviews.filter(col("id_review").startswith("R"))
filtered_reviews.select("id_review").show()

+---------+
|id_review|
+---------+
|    R1245|
|    R1246|
|    R1247|
|    R1248|
|    R1249|
|    R1250|
|    R1251|
|    R1252|
|    R1253|
|    R1254|
|    R1255|
|    R1256|
|    R1257|
|    R1258|
|    R1259|
|    R1260|
|    R1261|
|    R1262|
|    R1263|
|    R1264|
+---------+
only showing top 20 rows



In [77]:
from pyspark.sql.functions import col, lower, regexp_replace


filtered_reviews = filtered_reviews.withColumn("review", lower(col("review"))) #lower case
regex = "[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]"
filtered_reviews = filtered_reviews.withColumn("review", regexp_replace(col("review"), regex, ""))
filtered_reviews = filtered_reviews.withColumn("review", regexp_replace(col("review"), "\s+", " ")) #remove double spaces


# Remove duplicates
filtered_reviews = filtered_reviews.dropDuplicates(['id_review'])

In [78]:
filtered_reviews.select("id_review").show(5)

+---------+
|id_review|
+---------+
|    R3030|
|    R3673|
|    R3814|
|    R3940|
|    R4314|
+---------+
only showing top 5 rows



Drop missing values:

In [79]:
# Drop rows with missing reviews
filtered_reviews = filtered_reviews.na.drop(subset=["review"])
filtered_reviews = filtered_reviews.na.drop(subset=["service"])
filtered_reviews = filtered_reviews.na.drop(subset=["score"])

In [80]:
filtered_reviews.select("id_review").show(5)


+---------+
|id_review|
+---------+
| R1000000|
| R1000001|
| R1000003|
| R1000005|
| R1000006|
+---------+
only showing top 5 rows



Tokenize and Remove Stop Words:

In [81]:
# Load stop words in Spanish
!wget -O spanish_stop_words.txt https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt

--2024-02-25 04:19:56--  https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘spanish_stop_words.txt’

spanish_stop_words. 100%[===================>]   4.14K  --.-KB/s    in 0s      

2024-02-25 04:19:57 (22.2 MB/s) - ‘spanish_stop_words.txt’ saved [4244/4244]



In [82]:
# Read the stop words file into a list
with open('spanish_stop_words.txt', 'r') as file:
    spanish_stop_words = file.read().splitlines()

In [83]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Tokenize text
tokenizer = Tokenizer(inputCol="review", outputCol="words")
tokenized_df = tokenizer.transform(filtered_reviews)

# Remove stop words

remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=spanish_stop_words)
filtered_reviews = remover.transform(tokenized_df)

In [84]:
filtered_reviews.select('filtered_words').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+
|filtered_words                                                                                                          |
+------------------------------------------------------------------------------------------------------------------------+
|[]                                                                                                                      |
|[]                                                                                                                      |
|[come, parrillas]                                                                                                       |
|[excelente]                                                                                                             |
|[muñequitos]                                                                                                            |
|[comida, duda, 

In [85]:
# Getting rid of reviews with no words
from pyspark.sql.functions import size

# Filter out reviews where the 'filtered_words' array is empty
filtered_reviews = filtered_reviews.filter(size(col("filtered_words")) > 0)

# Show the filtered DataFrame to verify
filtered_reviews.select('filtered_words').show(10)

+--------------------+
|      filtered_words|
+--------------------+
|   [come, parrillas]|
|         [excelente]|
|        [muñequitos]|
|[comida, duda, es...|
|[pasar, momentos,...|
|[pollito, riquísimo]|
|[gente, nota, esf...|
|            [bonito]|
|[pasa, ensaladas,...|
|             [gusto]|
+--------------------+
only showing top 10 rows



In [86]:
filtered_reviews.columns

['id_review', 'review', 'service', 'score', 'words', 'filtered_words']

In [87]:
filtered_reviews.select('score').show(10)

+-----+
|score|
+-----+
|  5.0|
|  5.0|
|  3.0|
|  3.0|
|  5.0|
|  4.0|
|  3.0|
|  5.0|
|  2.0|
|  5.0|
+-----+
only showing top 10 rows



##Sentiment Analysis with 'Review' column

In [88]:
## I want to see the min and max values of the scores

from pyspark.sql.functions import min, max
# cast type as integer
from pyspark.sql.types import FloatType

filtered_reviews = filtered_reviews.withColumn("score", col("score").cast(FloatType()))


# Keep only the rows where 'score' is a numeric value
numeric_filtered_df = filtered_reviews.filter(col("score").isNotNull())
filtered = numeric_filtered_df.filter(col("score").rlike("^-?\\d+(\\.\\d+)?$"))

min_value = filtered.agg(min("score")).collect()[0][0]
max_value = filtered.agg(max("score")).collect()[0][0]

print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: 0.0
Maximum value: 5.0


In [89]:
from pyspark.sql.functions import when

# Define a threshold for positive sentiment
threshold = 3

# Create a new column 'sentiment_label' where scores greater than the threshold are labeled as positive (1)
# and scores less than or equal to the threshold are labeled as negative (0)
reviews_with_threshold = reviews.withColumn("sentiment_label", when(col("Score") > threshold, 1).otherwise(0))

In [90]:
reviews_with_threshold.select(col("sentiment_label")).show(10)

+---------------+
|sentiment_label|
+---------------+
|              0|
|              1|
|              1|
|              1|
|              0|
|              0|
|              0|
|              0|
|              1|
|              1|
+---------------+
only showing top 10 rows



# Sentiment Analysis with lexicon files

## **Afinn_df**

 Match Words to Lexicon Entries:

In [91]:
from pyspark.sql.functions import explode, col

In [92]:
exploded_reviews = filtered_reviews.withColumn("words", explode(col("filtered_words")))
joined_df = exploded_reviews.join(afinn_df, exploded_reviews.words == afinn_df.palabra, "left_outer")
joined_df = joined_df.dropDuplicates(['id_review'])


There's many reviews with mispelled words. I am going to drop those rows because I can't match them

In [93]:
joined_df = joined_df.na.drop(subset=["palabra"])

In [94]:
from pyspark.sql.functions import avg

average_sentiment_scores = joined_df.groupBy('id_review').agg(avg('punctuacion').alias('average_sentiment_score'))

#check for duplicates
average_sentiment_scores = average_sentiment_scores.dropDuplicates(['id_review'])


final_sentiment_scores = average_sentiment_scores.join(
    filtered_reviews.select('id_review', 'service'),
    on='id_review',
    how='left'
)

final_sentiment_scores.show()

+---------+-----------------------+--------+
|id_review|average_sentiment_score| service|
+---------+-----------------------+--------+
| R1000005|                    3.0|265714.0|
| R1000016|                    3.0|265714.0|
| R1000022|                    2.0|265714.0|
| R1000028|                    2.0|265714.0|
|  R100003|                    2.0| 40012.0|
| R1000050|                    2.0|265714.0|
| R1000051|                    3.0|265714.0|
| R1000053|                   -3.0|265714.0|
| R1000054|                    3.0|265714.0|
| R1000061|                    3.0|265714.0|
| R1000067|                    2.0|265714.0|
| R1000072|                    3.0|265714.0|
| R1000083|                    2.0|265714.0|
| R1000090|                   -3.0|265714.0|
| R1000095|                    3.0|265714.0|
| R1000107|                    2.0|265714.0|
| R1000112|                    2.0|265714.0|
|  R100022|                   -1.0| 40012.0|
|  R100033|                    2.0| estor G|
| R1000949

In [95]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

# Cast 'service' column to integer type to remove '.0'
final_sentiment_scores = final_sentiment_scores.withColumn("service", col("service").cast(IntegerType()))


In [96]:
final_sentiment_scores.columns

['id_review', 'average_sentiment_score', 'service']

In [97]:
## I want to see the min and max values of the scores


min_value = average_sentiment_scores.agg(min("average_sentiment_score")).collect()[0][0]
max_value = average_sentiment_scores.agg(max("average_sentiment_score")).collect()[0][0]

print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: -5.0
Maximum value: 5.0


## **Nrc_df**

In [98]:
joined_nrc_df = exploded_reviews.join(nrc_df, exploded_reviews.words == nrc_df.word, "left_outer")

from pyspark.sql.functions import col, count
emotion_counts_df = joined_nrc_df.groupBy("id_review").pivot("sentiment").count()
emotion_counts_df.show()

+---------+----+-------+------------+----+---------+------+-----+--------+--------+--------+--------+
|id_review|null|alegría|anticipación|asco|confianza|enfado|miedo|negativo|positivo|sorpresa|tristeza|
+---------+----+-------+------------+----+---------+------+-----+--------+--------+--------+--------+
| R1000005|NULL|      1|        NULL|NULL|        1|  NULL| NULL|    NULL|       1|    NULL|    NULL|
|  R100001|   6|      2|           1|   2|        2|     1|    3|       3|       4|       1|       2|
| R1000011|   5|      1|           2|NULL|        1|  NULL| NULL|    NULL|       1|    NULL|    NULL|
| R1000014|   2|   NULL|           1|NULL|        1|  NULL| NULL|    NULL|       4|    NULL|    NULL|
| R1000022|NULL|      1|        NULL|NULL|        1|  NULL| NULL|    NULL|       1|    NULL|    NULL|
| R1000027|   1|   NULL|        NULL|NULL|     NULL|  NULL|    1|    NULL|    NULL|    NULL|    NULL|
| R1000028|NULL|      1|        NULL|NULL|        1|  NULL| NULL|    NULL|       1

After looking at the result, I believe adding this analysis to the aggregate would not add much to the final result. Therefore, I will work solely with the Affin and Score analysis from now on.

# Agregating the Sentiment Analysis

In [99]:
# Normalize the sentiment score
final_sentiment_scores = final_sentiment_scores.withColumn(
    "normalized_score",
    ((col("average_sentiment_score") + 5) / 10)
)

# Show the results to verify
final_sentiment_scores.show()

+---------+-----------------------+-------+----------------+
|id_review|average_sentiment_score|service|normalized_score|
+---------+-----------------------+-------+----------------+
| R1000005|                    3.0| 265714|             0.8|
| R1000016|                    3.0| 265714|             0.8|
| R1000022|                    2.0| 265714|             0.7|
| R1000028|                    2.0| 265714|             0.7|
|  R100003|                    2.0|  40012|             0.7|
| R1000050|                    2.0| 265714|             0.7|
| R1000051|                    3.0| 265714|             0.8|
| R1000053|                   -3.0| 265714|             0.2|
| R1000054|                    3.0| 265714|             0.8|
| R1000061|                    3.0| 265714|             0.8|
| R1000067|                    2.0| 265714|             0.7|
| R1000072|                    3.0| 265714|             0.8|
| R1000083|                    2.0| 265714|             0.7|
| R1000090|             

In [100]:
# Normalize the rating score

final_rating_scores = filtered_reviews.withColumn(
    "normalized_score_r",
    (col("score") / 5)
)

final_rating_scores.show()

+---------+--------------------+--------+-----+--------------------+--------------------+------------------+
|id_review|              review| service|score|               words|      filtered_words|normalized_score_r|
+---------+--------------------+--------+-----+--------------------+--------------------+------------------+
| R1000003|se come bien parr...|265714.0|  5.0|[se, come, bien, ...|   [come, parrillas]|               1.0|
| R1000005|           excelente|265714.0|  5.0|         [excelente]|         [excelente]|               1.0|
| R1000006|    los muñequitos o|265714.0|  3.0|[los, muñequitos, o]|        [muñequitos]|               0.6|
|  R100001|la comida es buen...| 40012.0|  3.0|[la, comida, es, ...|[comida, duda, es...|               0.6|
| R1000011|se puede pasar bu...|265714.0|  5.0|[se, puede, pasar...|[pasar, momentos,...|               1.0|
| R1000013|el pollito riquísimo|265714.0|  4.0|[el, pollito, riq...|[pollito, riquísimo]|               0.8|
| R1000014|mucha ge

In [101]:
## Join the normalized scores

joined = final_sentiment_scores.join(
    final_rating_scores.select('id_review', 'normalized_score_r'),
    on='id_review',
    how='inner'
)

joined = joined.na.drop()

joined.show()

+---------+-----------------------+-------+----------------+------------------+
|id_review|average_sentiment_score|service|normalized_score|normalized_score_r|
+---------+-----------------------+-------+----------------+------------------+
| R1000016|                    3.0| 265714|             0.8|               1.0|
| R1000022|                    2.0| 265714|             0.7|               1.0|
| R1000028|                    2.0| 265714|             0.7|               0.4|
|  R100003|                    2.0|  40012|             0.7|               0.6|
| R1000050|                    2.0| 265714|             0.7|               0.8|
| R1000051|                    3.0| 265714|             0.8|               0.8|
| R1000053|                   -3.0| 265714|             0.2|               0.2|
| R1000061|                    3.0| 265714|             0.8|               1.0|
| R1000067|                    2.0| 265714|             0.7|               0.8|
| R1000072|                    3.0| 2657

In [102]:
joined = joined.withColumn(
    "average",
    ((col("normalized_score_r") +col("normalized_score"))/2)
)
joined.show()

+---------+-----------------------+-------+----------------+------------------+------------------+
|id_review|average_sentiment_score|service|normalized_score|normalized_score_r|           average|
+---------+-----------------------+-------+----------------+------------------+------------------+
| R1000016|                    3.0| 265714|             0.8|               1.0|               0.9|
| R1000022|                    2.0| 265714|             0.7|               1.0|              0.85|
| R1000028|                    2.0| 265714|             0.7|               0.4|              0.55|
|  R100003|                    2.0|  40012|             0.7|               0.6|0.6499999999999999|
| R1000050|                    2.0| 265714|             0.7|               0.8|              0.75|
| R1000051|                    3.0| 265714|             0.8|               0.8|               0.8|
| R1000053|                   -3.0| 265714|             0.2|               0.2|               0.2|
| R1000061

# Display the names of the restaurants with the best reviews

In [103]:
from pyspark.sql import functions as F

best_reviews = joined.join(
    restaurants,
    joined.service == restaurants.id
).orderBy(col('average').desc())

# select the restaurant names and sentiment scores.
best_reviews = best_reviews.filter(F.col('average') == 1)
best_reviews.show(10)

+---------+-----------------------+-------+----------------+------------------+-------+------+--------------------+-----------+
|id_review|average_sentiment_score|service|normalized_score|normalized_score_r|average|    id|                name|   district|
+---------+-----------------------+-------+----------------+------------------+-------+------+--------------------+-----------+
| R1017141|                    5.0| 124060|             1.0|               1.0|    1.0|124060|                Cala|   BARRANCO|
| R1015826|                    5.0|  28926|             1.0|               1.0|    1.0| 28926|LA BODEGA DE LA T...| SAN ISIDRO|
| R1003655|                    5.0| 113187|             1.0|               1.0|    1.0|113187|             Rustica| LOS OLIVOS|
| R1032797|                    5.0| 195070|             1.0|               1.0|    1.0|195070|San Antonio - Cha...|  SAN BORJA|
| R1058147|                    5.0|  42658|             1.0|               1.0|    1.0| 42658|   Callao 

In [65]:
best_reviews.count()

195

There are 195 restaurants in our **best_reviews** dataframe. I could choose randomly from these places, but in order to keep narrowing the list down, I will find out which places are near to our hotel, which is located in San Isidro district.

In [106]:
barranco_reviews = best_reviews.filter(F.col('district') == 'SAN ISIDRO')
barranco_reviews.show()

+---------+-----------------------+-------+----------------+------------------+-------+------+--------------------+----------+
|id_review|average_sentiment_score|service|normalized_score|normalized_score_r|average|    id|                name|  district|
+---------+-----------------------+-------+----------------+------------------+-------+------+--------------------+----------+
| R1184829|                    5.0| 187573|             1.0|               1.0|    1.0|187573|            Amoramar|SAN ISIDRO|
| R1015826|                    5.0|  28926|             1.0|               1.0|    1.0| 28926|LA BODEGA DE LA T...|SAN ISIDRO|
| R1163350|                    5.0| 165911|             1.0|               1.0|    1.0|165911|              Bembos|SAN ISIDRO|
|   R70889|                    5.0|  54216|             1.0|               1.0|    1.0| 54216|               Cosme|SAN ISIDRO|
| R1507214|                    5.0|  37214|             1.0|               1.0|    1.0| 37214|       Café A Bis

# Thoughts and ways of improvement

*In order to have a better sense of recommendation, it would've been useful to add a new category. Perhaps add a method to be able to filter restaurants by type of cuisine and/or price range.
 Finally,if there was a way to take care of those mispellings we would've had a much richer range of reviews, making more robust predictions on how good a restaurant is.*